## Get BaseClassifiers statistics

In [ ]:
import torch
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score
import joblib
import sys
import numpy as np
import json
import time
from sklearn.metrics import (
    accuracy_score, recall_score, f1_score, precision_score, roc_auc_score,
    top_k_accuracy_score
)


CLIP_FEATURES_DIR = "clip_features_new"
VAL = CLIP_FEATURES_DIR + "/val_features.pt"
SCALER = "scaler_model_new.joblib"
PCA = "pca_model_new.joblib"
LIME = "top_k_lime_indices.joblib"

In [7]:
def load_features(file_path):
    data = torch.load(file_path)
    return data["image_features"], data["text_features"], data["filenames"], data["labels"]


# Load train and validation features
val_img_features, val_txt_features, _, val_labels = load_features(VAL)

In [8]:
# Combine image and text features for training
X_val = torch.cat((val_img_features, val_txt_features), dim=1)

# Flatten features into a 2D matrix (samples x features)
X_val = X_val.view(X_val.size(0), -1).numpy()

# Print the shape of the features
print(f"X_val shape: {X_val.shape}")

# Convert labels to NumPy arrays
y_val = val_labels.numpy()  

# Load scaler and PCA models
scaler = joblib.load(SCALER)
pca = joblib.load(PCA)
lime = joblib.load(LIME)

# Scale and transform the features
X_val_scaled = scaler.transform(X_val)
X_val_pca = pca.transform(X_val_scaled)
X_val_lime = X_val_scaled[:, lime]


print(f"X_val_pca shape: {X_val_pca.shape}")
print(f"X_val_lime shape: {X_val_lime.shape}")

X_val shape: (1985, 1024)
X_val_pca shape: (1985, 563)
X_val_lime shape: (1985, 250)


In [ ]:
from classifiers import (
    SVMClassifier, RBFClassifier, RandomForestClassifier, NaiveBayesClassifier, 
    LogisticRegressionClassifier, LDAClassifier, KNNClassifier, DecisionTreeClassifier,
    AdaBoostClassifier, GBMClassifier, XGBoostClassifier
)

# Initialize classifiers
classifiers = {
    "SVM": SVMClassifier(),
    "RBF": RBFClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": NaiveBayesClassifier(),
    "Logistic Regression": LogisticRegressionClassifier(),
    "LDA": LDAClassifier(),
    "KNN": KNNClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GBMClassifier(),
    "XGBoost": XGBoostClassifier()
}

results = {}
models = ["models_pca_cal_sig_new"]

# Iterate over classifiers
for model in models:
    results[model] = {}

    # Choose input features based on model name
    X_val_input = X_val_pca if "pca" in model.lower() else X_val_lime

    for name, classifier in classifiers.items():
        print(f"\nEvaluating {name}...")
        try:
            classifier.load(model_dir=model)
            label_encoder = classifier.label_encoder
            all_labels = label_encoder.classes_

            # Time classification
            start_time = time.time()
            y_pred = classifier.classify(X_val_input)
            elapsed_time = time.time() - start_time

            # Check if classify_proba is available
            if hasattr(classifier, "classify_proba"):
                try:
                    y_proba = classifier.classify_proba(X_val_input)
                    top5_acc = top_k_accuracy_score(y_val, y_proba, k=5, labels=np.arange(1, 91))

                    # AUC-ROC computation — assumes multi-class OVR
                    auc_roc = roc_auc_score(y_val, y_proba, multi_class='ovr', average='weighted', labels=np.arange(1, 91))
                except Exception as e:
                    print(f"Probability-based metrics not available for {name}: {e}")
                    top5_acc = None
                    auc_roc = None
            else:
                top5_acc = None
                auc_roc = None

            acc = accuracy_score(y_val, y_pred)
            recall = recall_score(y_val, y_pred, average='weighted')
            f1 = f1_score(y_val, y_pred, average='weighted')
            precision = precision_score(y_val, y_pred, average='weighted')  # ← New metric

            results[model][name] = {
                "top-5 accuracy": top5_acc,
                "accuracy": acc,
                "recall": recall,
                "precision": precision,
                "f1-score": f1,
                "auc-roc": auc_roc,
                "inference_time_sec": elapsed_time
            }

            print(f"{name} -> Accuracy: {acc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Time: {elapsed_time:.4f}s")
            if top5_acc is not None:
                print(f"Top-5 Accuracy: {top5_acc:.4f}")
            if auc_roc is not None:
                print(f"AUC-ROC: {auc_roc:.4f}")

        except Exception as e:
            print(f"Failed to evaluate {name}: {e}")
            results[model][name] = {
                "top-5 accuracy": None,
                "accuracy": None,
                "recall": None,
                "precision": None,
                "f1-score": None,
                "auc-roc": None,
                "inference_time_sec": None,
                "error": str(e)
            }


Evaluating SVM...
Loaded model from: models_pca/SVM.joblib
Loaded label encoder from: models_pca/label_encoder.joblib


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defi

SVM -> Accuracy: 0.6247, F1: 0.6076, Precision: 0.6079, Recall: 0.6247, Time: 3.7794s
Top-5 Accuracy: 0.9123
AUC-ROC: 0.9377

Evaluating RBF...
Loaded model from: models_pca/RBF.joblib
Loaded label encoder from: models_pca/label_encoder.joblib


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defi

RBF -> Accuracy: 0.6448, F1: 0.5954, Precision: 0.5895, Recall: 0.6448, Time: 6.1943s
Top-5 Accuracy: 0.9174
AUC-ROC: 0.9401

Evaluating Random Forest...
Loaded model from: models_pca/RandomForest.joblib
Loaded label encoder from: models_pca/label_encoder.joblib


...

/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defi

AdaBoost -> Accuracy: 0.3572, F1: 0.1900, Precision: 0.1294, Recall: 0.3572, Time: 0.0399s
Top-5 Accuracy: 0.5678
AUC-ROC: 0.7150

Evaluating Gradient Boosting...
Loaded model from: models_lime/GBM.joblib
Loaded label encoder from: models_lime/label_encoder.joblib


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defi

Gradient Boosting -> Accuracy: 0.4343, F1: 0.4036, Precision: 0.4113, Recall: 0.4343, Time: 0.0654s
Top-5 Accuracy: 0.7451
AUC-ROC: 0.7907

Evaluating XGBoost...
Loaded model from: models_lime/XGBoost.joblib
Loaded label encoder from: models_lime/label_encoder.joblib
XGBoost -> Accuracy: 0.5798, F1: 0.5334, Precision: 0.5433, Recall: 0.5798, Time: 0.0328s
Top-5 Accuracy: 0.8635
AUC-ROC: 0.9235


/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/home/tiago/thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defi

In [ ]:
with open("test.json", "w") as f:
    json.dump(results, f, indent=4)

print("Results saved to classifier_results.json")

Results saved to classifier_results.json
